In [4]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

 ········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT f.title, c.name AS category, rental_duration, rental_rate, length, rating
FROM film f
JOIN inventory i
USING (film_id)
JOIN rental r
USING (inventory_id)
JOIN film_category fc
USING (film_id)
JOIN category c
USING (category_id);'''

data = pd.read_sql_query(query, engine)
data.head(50)

data.isna().value_counts()

title  category  rental_duration  rental_rate  length  rating
False  False     False            False        False   False     16044
dtype: int64

In [3]:
import statistics as stats

grouped_data = data.groupby('title').agg({
    'rental_duration': 'mean',
    'rental_rate': 'mean',
    'length': 'mean',
    'category': lambda x: stats.mode(x)[0][0],
    'rating': lambda x: stats.mode(x)[0][0]
}).reset_index()

print(grouped_data)

                 title  rental_duration  rental_rate  length category rating
0     ACADEMY DINOSAUR              6.0         0.99    86.0        D      P
1       ACE GOLDFINGER              3.0         4.99    48.0        H      G
2     ADAPTATION HOLES              7.0         2.99    50.0        D      N
3     AFFAIR PREJUDICE              5.0         2.99   117.0        H      G
4          AFRICAN EGG              6.0         2.99   130.0        F      G
..                 ...              ...          ...     ...      ...    ...
953     YOUNG LANGUAGE              6.0         0.99   183.0        D      G
954         YOUTH KICK              4.0         0.99   179.0        M      N
955       ZHIVAGO CORE              6.0         0.99   105.0        H      N
956  ZOOLANDER FICTION              5.0         2.99   101.0        C      R
957          ZORRO ARK              3.0         4.99    50.0        C      N

[958 rows x 6 columns]


In [5]:
query = '''SELECT f.title, c.name AS category, rental_duration, rental_rate, length, rating
FROM film f
JOIN film_category fc
USING (film_id)
JOIN category c
USING (category_id)
WHERE film_id IN (
    SELECT f.film_id
    FROM film f
    LEFT JOIN inventory i
    ON f.film_id = i.film_id
    WHERE i.film_id IS NULL
);;'''

missing_data = pd.read_sql_query(query, engine)
missing_data.head(50)

,title,category,rental_duration,rental_rate,length,rating
0,ARK RIDGEMONT,Action,6,0.99,68,NC-17
1,FIREHOUSE VIETNAM,Action,7,0.99,103,G
2,SKY MIRACLE,Action,7,2.99,132,PG
3,ARGONAUTS TOWN,Animation,7,0.99,127,PG-13
4,FLOATS GARDEN,Animation,6,2.99,145,PG-13
5,SISTER FREDDY,Children,5,4.99,152,PG-13
6,WALLS ARTIST,Children,7,4.99,135,PG
7,ALICE FANTASIA,Classics,6,0.99,94,NC-17
8,TADPOLE PARK,Classics,6,2.99,155,PG
9,VOLUME HOUSE,Classics,7,4.99,132,PG


In [6]:
data = pd.concat([grouped_data, missing_data], axis=0)

In [7]:
data.head()

,title,rental_duration,rental_rate,length,category,rating
0,ACADEMY DINOSAUR,6.0,0.99,86.0,D,P
1,ACE GOLDFINGER,3.0,4.99,48.0,H,G
2,ADAPTATION HOLES,7.0,2.99,50.0,D,N
3,AFFAIR PREJUDICE,5.0,2.99,117.0,H,G
4,AFRICAN EGG,6.0,2.99,130.0,F,G


In [8]:
data.value_counts()

title             rental_duration  rental_rate  length  category  rating
ACADEMY DINOSAUR  6.0              0.99         86.0    D         P         1
PERFECT GROOVE    7.0              2.99         82.0    C         P         1
PARK CITIZEN      3.0              4.99         109.0   A         P         1
PARTY KNOCK       7.0              2.99         107.0   C         P         1
PAST SUICIDES     5.0              4.99         157.0   F         P         1
                                                                           ..
FROGMEN BREAKING  5.0              0.99         111.0   T         R         1
FRONTIER CABIN    6.0              4.99         183.0   N         P         1
FROST HEAD        5.0              0.99         82.0    C         P         1
FUGITIVE MAGUIRE  7.0              4.99         83.0    T         R         1
ZORRO ARK         3.0              4.99         50.0    C         N         1
Length: 1000, dtype: int64

In [9]:
data.isna().value_counts()

title  rental_duration  rental_rate  length  category  rating
False  False            False        False   False     False     1000
dtype: int64

In [10]:
data.sort_values(by='title')
X = data.drop(['title'], axis = 1)

In [11]:
query = '''SELECT title,
       CASE
           WHEN MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005 THEN 1
           ELSE 0 END AS rented_in_May
FROM film f
JOIN inventory
USING (film_id)
JOIN rental r
USING (inventory_id);'''

data_target = pd.read_sql_query(query, engine)
data_target = data_target.groupby(['title']).agg({'rented_in_May':sum})

missing_films_query = ''' SELECT title 
FROM film 
WHERE film_id IN (
    SELECT f.film_id
    FROM film f
    LEFT JOIN inventory i
    ON f.film_id = i.film_id
    WHERE i.film_id IS NULL
);'''

missing_films = pd.read_sql_query(missing_films_query, engine)
missing_films['rented_in_May'] = 0
missing_films

,title,rented_in_May
0,ALICE FANTASIA,0
1,APOLLO TEEN,0
2,ARGONAUTS TOWN,0
3,ARK RIDGEMONT,0
4,ARSENIC INDEPENDENCE,0
5,BOONDOCK BALLROOM,0
6,BUTCH PANTHER,0
7,CATCH AMISTAD,0
8,CHINATOWN GLADIATOR,0
9,CHOCOLATE DUCK,0


In [12]:
data_target.reset_index(inplace = True)
missing_films.reset_index(inplace = True)

In [13]:
dt = pd.concat([data_target, missing_films], axis=0)

In [14]:
dt.drop(['index'], axis = 1)

,title,rented_in_May
0,ACADEMY DINOSAUR,2
1,ACE GOLDFINGER,0
2,ADAPTATION HOLES,1
3,AFFAIR PREJUDICE,2
4,AFRICAN EGG,1
...,...,...
37,TREASURE COMMAND,0
38,VILLAIN DESPERATE,0
39,VOLUME HOUSE,0
40,WAKE JAWS,0


In [15]:
dt = dt.sort_values(by='title')

In [16]:
y = dt

In [17]:
y

,title,rented_in_May,index
0,ACADEMY DINOSAUR,2,NaN
1,ACE GOLDFINGER,0,NaN
2,ADAPTATION HOLES,1,NaN
3,AFFAIR PREJUDICE,2,NaN
4,AFRICAN EGG,1,NaN
...,...,...,...
953,YOUNG LANGUAGE,0,NaN
954,YOUTH KICK,0,NaN
955,ZHIVAGO CORE,1,NaN
956,ZOOLANDER FICTION,1,NaN


In [18]:
X

,rental_duration,rental_rate,length,category,rating
0,6.0,0.99,86.0,D,P
1,3.0,4.99,48.0,H,G
2,7.0,2.99,50.0,D,N
3,5.0,2.99,117.0,H,G
4,6.0,2.99,130.0,F,G
...,...,...,...,...,...
37,7.0,4.99,99.0,Sports,NC-17
38,4.0,0.99,137.0,Travel,PG
39,7.0,0.99,76.0,Travel,NC-17
40,3.0,4.99,53.0,Travel,NC-17


In [19]:
# I am going to treat rental_rate and rental_duration as category

# X['rental_rate'] = X['rental_rate'].astype(str)
# X['rental_duration'] = X['rental_duration'].astype(str)

In [20]:
X.dtypes

rental_duration    float64
rental_rate        float64
length             float64
category            object
rating              object
dtype: object

In [22]:
import numpy as np

X_num = X.select_dtypes(include = np.number)

In [23]:
# here we start scaling, so need to do train-test-split before
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [24]:
X_train_num = X_train.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include=[np.object])


X_test_num = X_test.select_dtypes(include = np.number)
X_test_cat = X_test.select_dtypes(include=[np.object])



/var/folders/n7/3f1mct1521j5vz1r2r1xwby80000gn/T/ipykernel_8830/1781428393.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_cat = X_train.select_dtypes(include=[np.object])
/var/folders/n7/3f1mct1521j5vz1r2r1xwby80000gn/T/ipykernel_8830/1781428393.py:6: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_cat = X_test.select_dtypes(include=[np.object])


In [27]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_num.columns)
X_train_norm

,rental_duration,rental_rate,length
0,1.00,2.775558e-17,0.280576
1,1.00,2.775558e-17,0.050360
2,1.00,1.000000e+00,0.187050
3,0.25,2.775558e-17,0.438849
4,0.75,1.000000e+00,0.410072
...,...,...,...
745,0.00,5.000000e-01,1.000000
746,1.00,1.000000e+00,0.539568
747,0.50,5.000000e-01,0.014388
748,0.25,0.000000e+00,0.424460


In [28]:
transformer = MinMaxScaler().fit(X_test_num) # need to keep transformer
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized, columns=X_num.columns)
X_test_norm

,rental_duration,rental_rate,length
0,0.00,1.000000e+00,0.474820
1,0.00,1.000000e+00,0.956835
2,0.25,0.000000e+00,0.791367
3,0.00,1.000000e+00,0.741007
4,0.75,1.000000e+00,0.388489
...,...,...,...
245,1.00,1.000000e+00,0.093525
246,1.00,2.775558e-17,0.640288
247,1.00,5.000000e-01,0.028777
248,0.50,2.775558e-17,0.000000


In [29]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first').fit(X_train_cat)

cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
X_train_encoded = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)
X_train_encoded

,category_Action,category_Animation,category_C,category_Children,category_Classics,category_Comedy,category_D,category_Documentary,category_F,category_Family,...,category_Sci-Fi,category_Sports,category_T,category_Travel,rating_N,rating_NC-17,rating_P,rating_PG,rating_PG-13,rating_R
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
747,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
748,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X_test_cat

,category,rating
35,Sci-Fi,R
859,D,G
298,F,R
553,A,P
672,A,P
...,...,...
462,F,R
356,F,G
2,D,N
478,D,P


In [33]:
X_train_cat

,category,rating
253,N,N
667,A,G
85,H,N
11,Comedy,PG
75,S,P
...,...,...
835,T,R
192,F,P
629,C,P
559,D,G


In [31]:
cols = encoder.get_feature_names_out(input_features=X_test_cat.columns)
X_test_encoded = pd.DataFrame(encoder.transform(X_test_cat).toarray(),columns=cols)
X_test_encoded

ValueError: Found unknown categories ['Drama', 'New'] in column 0 during transform

In [ ]:
transformer = StandardScaler().fit(X_train)
X_train_scaled = pd.DataFrame(transformer.transform(X_train),columns=X.columns)
# because this is the only tranformation we apply to the numerics, 
# we can immediately transform the X_test as well
X_test_scaled = pd.DataFrame(transformer.transform(X_test),columns=X.columns)
X_train_scaled.head()

In [ ]:
# Now, that I have the basis of my X, I will encode the categories values

from sklearn.preprocessing import OneHotEncoder

categorical_columns = X.select_dtypes(include=['object', 'category'])

encoder = OneHotEncoder().fit(categorical_columns)
cols = encoder.get_feature_names_out(input_features=categorical_columns.columns)
categorical_columns_encoded = pd.DataFrame(encoder.transform(categorical_columns).toarray(),columns=cols)
categorical_columns_encoded

In [ ]:
numerical_colums = X.select_dtypes(include=['int64', 'float64'])

numerical_colums.head()
numerical_colums.shape

# new_X = pd.concat([numerical_coulms,categorical_columns_encoded], axis=1)

# X.head()

In [ ]:
categorical_columns_encoded

In [ ]:
import numpy as np

new_X = np.concatenate([numerical_colums,categorical_columns_encoded], axis=1)

In [ ]:
new_X

In [ ]:
numerical_colums.columns

In [ ]:
categorical_columns_encoded.columns